## San Francisco crime data modeling & analysis

#### Use PySpark DataFrame to Manipulate Dataset
(https://data.sfgov.org/Public-Safety/sf-data/skgt-fej3/data)

Reference: Chicago Crime
https://datascienceplus.com/spark-dataframes-exploring-chicago-crimes/

https://www.analyticsvidhya.com/blog/2016/10/spark-dataframe-and-operations/

In [3]:
from csv import reader
from pyspark.sql import Row 
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import to_date, to_timestamp, year, month, dayofmonth, hour, minute
from pyspark.sql.functions import udf, lit
import pyspark.sql.functions as fn
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from ggplot import *
import warnings
import math

import os
os.environ["PYSPARK_PYTHON"] = "python3"


In [4]:
#download data from SF gov's official website
#import urllib.request
#urllib.request.urlretrieve("https://data.sfgov.org/api/views/tmnf-yvry/rows.csv?accessType=DOWNLOAD", "/tmp/sf_03_18.csv")
#dbutils.fs.mv("file:/tmp/sf_03_18.csv", "dbfs:/laioffer/spark_hw1/data/sf_03_18.csv")
#display(dbutils.fs.ls("dbfs:/laioffer/spark_hw1/data/"))

#or download the file locally
#https://data.sfgov.org/api/views/tmnf-yvry/rows.csv?accessType=DOWNLOAD

In [5]:
data_path = "dbfs:/laioffer/spark_hw1/data/sf_03_18.csv"

### Use pyspark dataframe to manipulate dataset
RDD is registered to the dataframe

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("crime analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [8]:
df = spark.read.format("csv").option("header", "true").load(data_path)

In [9]:
df = df.drop(*[s for s in df.columns if s.startswith(":@")]) # drop all the unusful columns

#### Q1 question (OLAP): 
#####Write a Spark program that counts the number of crimes for different category.

In [11]:
q1_result = df.groupBy('Category').count().orderBy('count', ascending=False)
display(q1_result)

Category,count
LARCENY/THEFT,480448
OTHER OFFENSES,309358
NON-CRIMINAL,238323
ASSAULT,194694
VEHICLE THEFT,126602
DRUG/NARCOTIC,119628
VANDALISM,116059
WARRANTS,101379
BURGLARY,91543
SUSPICIOUS OCC,80444


#### Conclusions for Q1:
- The most prevelant crime type is Larceny/Theft

#### Q2 question (OLAP)
Counts the number of crimes for different district, and visualize your results

In [14]:
q2_result = df.groupBy('PdDistrict').count().orderBy('count', ascending=False)
display(q2_result)

PdDistrict,count
SOUTHERN,399785
MISSION,300076
NORTHERN,272713
CENTRAL,226255
BAYVIEW,221000
INGLESIDE,194180
TENDERLOIN,191746
TARAVAL,166971
PARK,125479
RICHMOND,116818


#### Conclusions for Q2:
- The districts with the most crime count are: Southern, Mission and Northern.
- It would be interesting to consider the population of each district as well.

#### Q3 question (OLAP)
Count the number of crimes each "Sunday" at "SF downtown".

In [17]:
# string to date format: https://stackoverflow.com/a/41273036/7065092
df = df.withColumn("Date", to_date(df.Date, format = "MM/dd/yyyy"))

In [18]:
# cast coordinate from string to float
df = df \
      .withColumn("Lat", df.Y.cast(FloatType())) \
      .withColumn("Lon", df.X.cast(FloatType()))

In [19]:
# We define "SF downtown" as 1.5 km within the Montgomery subway station

downtownCenter = (37.784824, -122.407525) # (lat, lon) coordinate of the Montgomery subway station
downtownCenterLat, downtownCenterLon = downtownCenter
downtownRadius = 1.5 # km

In [20]:
def haversine(lat1, lon1, lat2 = downtownCenterLat, lon2 = downtownCenterLon):
    # calculate distance between two locations
    # given (lat, lon) of two locations, in degree
    # return distance, in km, float type
    lat1, lon1, lat2, lon2 = np.radians((lat1, lon1, lat2, lon2))
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return float(6367 * c)

haversine_udf = udf(haversine, FloatType())  # udf

In [21]:
df = df.withColumn("DistToCenter", haversine_udf(df.Lat, df.Lon)) # calculate distance to city center

In [22]:
q3_result = df.filter(df.DayOfWeek == "Sunday") \
              .filter(df.DistToCenter < downtownRadius) \
              .groupBy('Date').count() \
              .withColumnRenamed("count", "downtownCount")\
              .orderBy('Date')
display(q3_result)

Date,downtownCount
2003-01-05,94
2003-01-12,110
2003-01-19,90
2003-01-26,104
2003-02-02,138
2003-02-09,120
2003-02-16,127
2003-02-23,104
2003-03-02,112
2003-03-09,120


In [23]:
# calculate crime count in all area, and join with downtown area count
q3_result = q3_result.join(
                df.filter(df.DayOfWeek == "Sunday") \
                  .groupBy('Date').count() \
                  .withColumnRenamed("count", "allAreaCount"),
                "Date"
              ) 

# calculate the percentage of downtownCrime vs. allCrime, on every Sunday
q3_result = q3_result.withColumn("downtownPercent", q3_result.downtownCount / q3_result.allAreaCount)

In [24]:
display(q3_result)

Date,downtownCount,allAreaCount,downtownPercent
2005-01-16,121,365,0.3315068493150685
2006-05-21,95,358,0.26536312849162014
2009-11-22,114,288,0.3958333333333333
2004-10-24,134,371,0.3611859838274933
2006-04-23,131,371,0.353099730458221
2009-06-28,142,382,0.3717277486910995
2009-08-09,128,346,0.3699421965317919
2009-09-27,128,358,0.3575418994413408
2011-01-30,115,313,0.36741214057507987
2003-12-07,88,289,0.3044982698961938


#### conclusions of Q3

- We made two time series: downtown crime every Sunday, and downtown crime percentage (vs. all SF).
- The high spikes are probably related to some special events, such as parade or gathering.
- In the downtown crime **count** plot, there is no obvious long-term trend over time.
- In the downtown crime **percentage** plot, we can see an increaseing trend from 2006 to 2009, and a decreasing trend from 2009 to 2012. 
- If we want to do it very carefully, we can do statistical tests about the trend and slope. Here we're just doing EDA to identify potentially interesting stories.

#### Q4 question (OLAP)
Analysis the number of crime in each month of 2015, 2016, 2017, 2018. Then, give your insights for the output results. What is the business impact for your result?

In [27]:
display(df.groupBy(month(df.Date), year(df.Date)).count())

month(Date),year(Date),count
1,2004,13086
12,2007,10886
6,2016,12094
1,2017,13084
1,2016,12967
8,2012,12333
5,2009,11364
10,2008,12941
2,2014,11565
12,2013,11242


In [28]:
display(df.filter((year(df.Date) < 2018) & (year(df.Date) >= 2015)).groupBy(month(df.Date), year(df.Date)).count())

month(Date),year(Date),count
6,2016,12094
1,2017,13084
1,2016,12967
3,2017,13711
11,2016,12720
2,2016,12106
6,2017,12605
8,2016,12471
3,2015,13929
4,2016,12328


#### conclusions of Q4:

- From the previous plots, we can see a strong seasonality effect in the monthly crime event time series. 
  - Most significantly, crime event count drops in every Feburary. Speculatively, this might be related to the lower temperature.

- For potential visitors, Feburary might be a good time of year to visit, considering the lower frequency of crimes. 

- For business owners in SF, consider cautiously lowering security budget for Feburary. 

- For police forces and policy makers, it would be useful to find out the reasons behind the seasonality of crime rates, and adjust the policies accordingly. Understanding the reasons behind the seasonality requires comparision with other datasets, such as temperature data and tourism data, and is beyond the scope of this project.

#### Q5 question (OLAP)
Analysis the number of crime w.r.t the hour in certian day like 2015/12/15, 2016/12/15, 2017/12/15. Then, give your travel suggestion to visit SF.

In [31]:
# convert Time from string to timestamp type
df = df.withColumn("Time", to_timestamp(df.Time, format = "HH:mm"))

In [32]:
display(
  df.filter(
    (df.Date == lit("2015-12-15")) | (df.Date == lit("2016-12-15")) | (df.Date == lit("2017-12-15"))
  ).groupBy(hour(df.Time), df.Date).count()
)

hour(Time),Date,count
3,2017-12-15,4
16,2016-12-15,17
18,2016-12-15,38
5,2015-12-15,3
17,2016-12-15,20
15,2017-12-15,27
13,2016-12-15,18
0,2016-12-15,22
4,2015-12-15,10
17,2017-12-15,28


In [33]:
display(
  df\
  .filter(df.DistToCenter < downtownRadius) \
  .groupBy(hour(df.Time)).count()
)

hour(Time),count
12,43499
22,37848
1,23957
13,39971
16,44341
6,12854
3,13142
20,36727
5,8346
19,42194


#### Conclusion for Q5:
- As can be seen in these two plots, 
  - There are fewer crime events after midnight, and the lowest crime count hours are around 5 am.
  - Hourly crime count is lower in the morning and before noon. Hourly crime count is higher in the afternoon and in the evening.
- Suggestion to visitors:
  - SF is safer in the morning than in the afternoon and evening. 
  - Pay more attention to protect your property and yourself after noon.

#### Q6 question (OLAP)
(1) Step1: Find out the top-3 danger district  
(2) Step2: find out the crime event w.r.t category and time (hour) from the result of step 1  
(3) give your advice to distribute the police based on your analysis results.

In [36]:
display(df.groupBy("PdDistrict").count().orderBy("count", ascending = False))

PdDistrict,count
SOUTHERN,399785
MISSION,300076
NORTHERN,272713
CENTRAL,226255
BAYVIEW,221000
INGLESIDE,194180
TENDERLOIN,191746
TARAVAL,166971
PARK,125479
RICHMOND,116818


In [37]:
display(df.groupBy("PdDistrict").count().orderBy("count", ascending = False).limit(3))

PdDistrict,count
SOUTHERN,399785
MISSION,300076
NORTHERN,272713


In [38]:
display(
  df \
  .filter((df.PdDistrict == "SOUTHERN") | (df.PdDistrict == "MISSION") | (df.PdDistrict == "NORTHERN")) \
  .groupBy(df.Category) \
  .count() \
  .orderBy("count", ascending = False)
)

Category,count
LARCENY/THEFT,243290
OTHER OFFENSES,129739
NON-CRIMINAL,108217
ASSAULT,81163
DRUG/NARCOTIC,50903
WARRANTS,49254
VANDALISM,46032
VEHICLE THEFT,43573
BURGLARY,36500
SUSPICIOUS OCC,31785


In [39]:
display(
  df \
  .filter((df.PdDistrict == "SOUTHERN") | (df.PdDistrict == "MISSION") | (df.PdDistrict == "NORTHERN")) \
  .groupBy(hour(df.Time), df.PdDistrict) \
  .count()
)

hour(Time),PdDistrict,count
2,NORTHERN,7809
16,MISSION,16296
19,NORTHERN,16503
7,SOUTHERN,9379
8,NORTHERN,9535
1,SOUTHERN,11203
3,SOUTHERN,5712
3,MISSION,5599
9,SOUTHERN,16608
23,SOUTHERN,18294


In [40]:
display(
  df \
  .filter(df.PdDistrict == "SOUTHERN") \
  .groupBy(hour(df.Time), df.Category) \
  .count() \
  .orderBy(hour(df.Time), 'count', df.Category, ascending = [True, False, True])
)

hour(Time),Category,count
0,LARCENY/THEFT,4163
0,OTHER OFFENSES,2947
0,NON-CRIMINAL,2414
0,ASSAULT,1714
0,FRAUD,1174
0,VANDALISM,1052
0,SUSPICIOUS OCC,923
0,FORGERY/COUNTERFEITING,858
0,WARRANTS,749
0,DRUG/NARCOTIC,648


In [41]:
display(
  df \
  .filter(df.PdDistrict == "MISSION") \
  .groupBy(hour(df.Time), df.Category) \
  .count() \
  .orderBy(hour(df.Time), 'count', df.Category, ascending = [True, False, True])
)

hour(Time),Category,count
0,OTHER OFFENSES,2754
0,LARCENY/THEFT,2554
0,ASSAULT,1801
0,NON-CRIMINAL,1538
0,VANDALISM,868
0,PROSTITUTION,732
0,SUSPICIOUS OCC,687
0,WARRANTS,677
0,DRUG/NARCOTIC,641
0,FRAUD,619


In [42]:
display(
  df \
  .filter(df.PdDistrict == "NORTHERN") \
  .groupBy(hour(df.Time), df.Category) \
  .count() \
  .orderBy(hour(df.Time), 'count', df.Category, ascending = [True, False, True])
)

hour(Time),Category,count
0,LARCENY/THEFT,3376
0,OTHER OFFENSES,2048
0,NON-CRIMINAL,1271
0,ASSAULT,1209
0,VANDALISM,761
0,FRAUD,708
0,BURGLARY,618
0,SUSPICIOUS OCC,581
0,WARRANTS,527
0,DRUG/NARCOTIC,486


In [43]:
df_q6 = df \
        .groupBy(hour(df.Time), df.PdDistrict, df.Category) \
        .count() \
        .withColumnRenamed("count", "categoryCount") \
        .join(
          df\
            .groupBy(hour(df.Time), df.PdDistrict)\
            .count(),
          on = ["PdDistrict", "hour(Time)"]
         ) \
         .withColumn("categoryPercent", fn.col("categoryCount") / fn.col("count")) 

In [44]:
display(df_q6
        .filter((df_q6.PdDistrict == "SOUTHERN") | (df_q6.PdDistrict == "MISSION") | (df_q6.PdDistrict == "NORTHERN")) \
        .filter(df_q6.Category == "LARCENY/THEFT")
       )

PdDistrict,hour(Time),Category,categoryCount,count,categoryPercent
NORTHERN,2,LARCENY/THEFT,1278,7809,0.16365731847867845
MISSION,16,LARCENY/THEFT,2604,16296,0.15979381443298968
NORTHERN,19,LARCENY/THEFT,6819,16503,0.4131976004362843
SOUTHERN,7,LARCENY/THEFT,1605,9379,0.17112698581938374
NORTHERN,8,LARCENY/THEFT,2068,9535,0.21688515993707394
SOUTHERN,1,LARCENY/THEFT,2577,11203,0.23002767115951084
SOUTHERN,3,LARCENY/THEFT,952,5712,0.16666666666666666
MISSION,3,LARCENY/THEFT,649,5599,0.11591355599214145
SOUTHERN,9,LARCENY/THEFT,3633,16608,0.21875
SOUTHERN,23,LARCENY/THEFT,5669,18294,0.30988302175576693


In [45]:
display(df_q6
        .filter((df_q6.PdDistrict == "SOUTHERN") | (df_q6.PdDistrict == "MISSION") | (df_q6.PdDistrict == "NORTHERN")) \
        .filter(df_q6.Category == "ASSAULT")
       )

PdDistrict,hour(Time),Category,categoryCount,count,categoryPercent
NORTHERN,2,ASSAULT,993,7809,0.12716096811371494
MISSION,16,ASSAULT,1479,16296,0.09075846833578792
NORTHERN,19,ASSAULT,985,16503,0.059686117675574135
SOUTHERN,7,ASSAULT,724,9379,0.07719373067491203
NORTHERN,8,ASSAULT,716,9535,0.07509176717357105
SOUTHERN,1,ASSAULT,1506,11203,0.13442827813978397
SOUTHERN,3,ASSAULT,697,5712,0.12202380952380952
MISSION,3,ASSAULT,644,5599,0.11502053938203251
SOUTHERN,9,ASSAULT,1163,16608,0.07002649325626205
SOUTHERN,23,ASSAULT,1398,18294,0.0764184978681535


#### Conclusions for Q6:
- For these three districts, the hourly crime count increases from 5 am, and has two peaks at noon and 6 pm. The hourly crime count significantly decereases from midnight to 5 am. 
- Larceny/Theft is the most prevelant crime category, and its percentage increases continiously from around 4-5 am to 7-8 pm. 
- Theft percentage is much higher in Northern and Southern Districts (as high as ~40% in evenings), than in Mission District (as high as ~20% in evenings). 
- Assault percentage has a peak at around 1-2 am. 
- Suggestions for police force assignment:
  - Overall, police force should be more focused on noon to midnight time, when hourly crime count is higher
  - Theft is the most prevelant crime in these three districts. Police force should be well-prepared to handle theft events and pay more attention to suspecious activities, especially around evening, and in Northern and Southern districts. Consider assigning theft-crime specialized police officers to aforementioned time and districts.
  - Assault event has a spike in percentage at around 1-2 am, when around 10% - 15% of crime events are assults. Police officers should be more alerted and prepared to handle assult cases.
- Similar analysis can be done for other categories as well

#### Q7 question (OLAP)
For different category of crime, find the percentage of resolution. Based on the output, give your hints to adjust the policy.

In [48]:
display(df.groupBy(df.Resolution).count().orderBy("count", ascending = False))

Resolution,count
NONE,1389500
"ARREST, BOOKED",524979
"ARREST, CITED",154789
LOCATED,34463
PSYCHOPATHIC CASE,29185
UNFOUNDED,23799
JUVENILE BOOKED,14158
COMPLAINANT REFUSES TO PROSECUTE,8089
DISTRICT ATTORNEY REFUSES TO PROSECUTE,7955
NOT PROSECUTED,7720


If and only if the `resolution` is `NONE` or `UNFOUNDED`, then we categorize this event as "not resolved".

In [50]:
df_q7 = df.groupBy(df.Category).count().orderBy(df.Category).withColumnRenamed("count", "totalCount")

df_q7 = df_q7.join(
            df.filter((df.Resolution != "NONE") & (df.Resolution != "UNFOUNDED"))\
              .groupBy(df.Category).count()\
              .withColumnRenamed("count", "resolvedCount"), 
           "Category")

df_q7 = df_q7.withColumn("resolvedPercent", df_q7.resolvedCount / df_q7.totalCount)

In [51]:
display(df_q7.orderBy(df_q7.resolvedPercent, ascending = False))

Category,totalCount,resolvedCount,resolvedPercent
PROSTITUTION,16701,15826,0.9476079276690018
WARRANTS,101379,95765,0.9446236400043402
DRIVING UNDER THE INFLUENCE,5672,5349,0.9430535966149506
DRUG/NARCOTIC,119628,109138,0.9123114989801718
LIQUOR LAWS,4083,3629,0.8888072495713936
STOLEN PROPERTY,11891,10435,0.8775544529476075
LOITERING,2430,2128,0.8757201646090536
DRUNKENNESS,9826,8073,0.8215957663342154
WEAPON LAWS,22234,16008,0.7199784114419358
OTHER OFFENSES,309358,219784,0.7104519682697716


In [52]:
display(df_q7.orderBy(df_q7.totalCount, ascending = False).limit(10))

Category,totalCount,resolvedCount,resolvedPercent
LARCENY/THEFT,480448,41300,0.08596143599307313
OTHER OFFENSES,309358,219784,0.7104519682697716
NON-CRIMINAL,238323,46061,0.1932713166584845
ASSAULT,194694,79672,0.4092165141195928
VEHICLE THEFT,126602,5833,0.04607352174531208
DRUG/NARCOTIC,119628,109138,0.9123114989801718
VANDALISM,116059,13732,0.11831913078692734
WARRANTS,101379,95765,0.9446236400043402
BURGLARY,91543,14345,0.1567023147591842
SUSPICIOUS OCC,80444,6885,0.08558748943364328


#### Conclusion for Q7

- As can be seen in the plot and table, some very prevelant crime category have very low resolution percentage, especially Larceny/Theft and Vehicle Theft
- Policy makers and police forces should consider methods that might improve the resolution percentage of these two types of crimes. Such as educating the citizens some ways to protect their personal items and vehicles from theft, and installing tracking devices for vehicles.

### Conclusion/Summary

- In order to better understand the crime events in San Francisco, and provide advice to police makers, police forces and visitors, we used Apache Spark distributed computing framework and analyzed over two million rows of crime event data from San Francisco government. 
- We used Spark Dataframe to organize and analyze the data, and performed a series of data visualization and OLAP to analyzed the spatial and temporal distributions of crime events, as well as the categories of the most prevelant crimes and resolution percentage.
- For more detailed conclusions, refer to the plots and conclusions of each question.
- Here are some main points:
  - Temporally, crime counts are lower in colder months (esp. Feburary), and lower after midnight and before dawn (around 5 am).
  - Spatially, crime counts are higher in Southern, Mission and Northern PD districts. Theft accounts for very high percentage of crimes in Southern and Northern districts.
  - The most prevelant crime type, Larceny/Theft, has very significant temporal and spatial distribution patterns. 
  - The resolution rates of some most prevelant crime categories are very low, especially for Theft and Vehicle Theft. 
  - Police forces and policy makers should adjust the security plans and policies based on these information. Visitors should also adjust their travel plans and use cautions as well

#### Some Caveats and further study ideas
- Higher crime count could mean actually more crimes, or more police officers on the street that can record these crime events
- Crime count can be adjusted by population in an area
- Data quality of time: police officers tend to round the time to the nearest 5min, 10min, 30min, 1 hour time. It would be interesting to make a model and reconstruct the real temporal distribution of crime events
- Time of crime event actually happening vs. crime being reported or noticed are different. For example, one possible explanation of higher theft rate at evening might be that tourists didn't discover they lost some items until dinner/supper time. Of course, these speculations need to be supported by additional data and further studies

In [56]:
display(
  df \
  .groupBy(df.Time).count()
)

Time,count
1970-01-01T00:14:00.000+0000,548
1970-01-01T16:06:00.000+0000,625
1970-01-01T12:55:00.000+0000,1973
1970-01-01T18:09:00.000+0000,682
1970-01-01T15:31:00.000+0000,552
1970-01-01T06:53:00.000+0000,291
1970-01-01T17:40:00.000+0000,3747
1970-01-01T06:30:00.000+0000,5140
1970-01-01T03:16:00.000+0000,298
1970-01-01T17:24:00.000+0000,755


In [57]:
display(
  df \
  .groupBy(fn.minute(df.Time)).count() \
  .orderBy("count", ascending = False)
)

minute(Time),count
0,676161
30,315063
45,100378
15,93838
1,66766
20,65444
50,63736
40,61546
10,56438
25,39980
